# AGGREGATING DATAFRAME

In [2]:
import findspark

In [3]:
findspark.init()

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [5]:
spark = SparkSession.builder.appName('AGGREGTING DATA FRAME').getOrCreate()
spark

In [6]:
df = spark.read.csv('nyc_air_bnb.csv',header=True,inferSchema=True)

In [8]:
df.limit(5).toPandas()

C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.9419,Private room,150,3,0,None,None,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [9]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: string (nullable = true)
 |-- minimum_nights: string (nullable = true)
 |-- number_of_reviews: string (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: string (nullable = true)
 |-- calculated_host_listings_count: string (nullable = true)
 |-- availability_365: integer (nullable = true)



In [10]:
from pyspark.sql.types import IntegerType,FloatType

In [11]:
clean_df = df.withColumn('id',df['id'].cast(IntegerType())).withColumn('host_id',df['host_id'].cast(IntegerType()))\
             .withColumn('latitude',df['latitude'].cast(FloatType())).withColumn('longitude',df['longitude'].cast(FloatType()))\
             .withColumn('price',df['price'].cast(IntegerType())).withColumn('minimum_nights',df['minimum_nights'].cast(IntegerType()))\
             .withColumn('number_of_reviews',df['number_of_reviews'].cast(IntegerType()))\
             .withColumn('last_review',to_date(df['last_review'],'yyyy-mm-dd'))\
             .withColumn('reviews_per_month',df['reviews_per_month'].cast(FloatType()))\
             .withColumn('calculated_host_listings_count',df['calculated_host_listings_count'].cast(IntegerType()))

In [12]:
clean_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: integer (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: float (nullable = true)
 |-- longitude: float (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- minimum_nights: integer (nullable = true)
 |-- number_of_reviews: integer (nullable = true)
 |-- last_review: date (nullable = true)
 |-- reviews_per_month: float (nullable = true)
 |-- calculated_host_listings_count: integer (nullable = true)
 |-- availability_365: integer (nullable = true)



In [13]:
clean_df.limit(5).toPandas()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.647491,-73.972366,Private room,149,1,9,2018-01-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.753620,-73.983772,Entire home/apt,225,1,45,2019-01-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.809021,-73.941902,Private room,150,3,0,None,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.685139,-73.959763,Entire home/apt,89,1,270,2019-01-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.798512,-73.943993,Entire home/apt,80,10,9,2018-01-19,0.10,1,0


In [17]:
clean_df.groupBy("neighbourhood_group").count().show(7)

+-------------------+-----+
|neighbourhood_group|count|
+-------------------+-----+
|         Douglaston|    1|
|             Queens| 5630|
|              Nadia|    1|
|            Midtown|    4|
|    Jackson Heights|    2|
|     Hell's Kitchen|    7|
|  Greenwich Village|    2|
+-------------------+-----+
only showing top 7 rows



In [18]:
clean_df.groupBy('neighbourhood_group').min('price').show(5)

+-------------------+----------+
|neighbourhood_group|min(price)|
+-------------------+----------+
|         Douglaston|         1|
|             Queens|        10|
|              Nadia|      null|
|            Midtown|         2|
|    Jackson Heights|         2|
+-------------------+----------+
only showing top 5 rows



In [30]:

list_summary = clean_df.select("price","number_of_reviews","minimum_nights").summary('min','25%','50%','75%','max')

In [31]:
list_summary.limit(5).toPandas()

,summary,price,number_of_reviews,minimum_nights
0,min,-74,0,0
1,25%,69,1,1
2,50%,105,5,3
3,75%,175,23,5
4,max,10000,629,1250


In [34]:
clean_df.select(countDistinct('neighbourhood_group').alias('Groups'),avg('price').alias('avgPrice'),stddev('price').alias('VariationInPrice')).limit(5).toPandas()

,Groups,avgPrice,VariationInPrice
0,77,152.222984,238.541467


In [42]:
clean_df.filter("room_type='Shared room'").groupBy("room_type").pivot("neighbourhood_group",["Brooklyn","Manhattan"]).agg(min('price'),max('price'),count('room_type')).toPandas()

,room_type,Brooklyn_min(price),Brooklyn_max(price),Brooklyn_count(room_type),Manhattan_min(price),Manhattan_max(price),Manhattan_count(room_type)
0,Shared room,0,725,413,10,1000,478


In [50]:
clean_df.groupBy('host_name').agg(sumDistinct('number_of_reviews'),max('number_of_reviews')).toPandas()

,host_name,sum(DISTINCT number_of_reviews),max(number_of_reviews)
0,Alayna,28.0,27.0
1,Faye,152.0,56.0
2,Leoluis,24.0,24.0
3,Paul & Elena,25.0,25.0
4,Mary D,5.0,5.0
...,...,...,...
11426,Angelica And Prescott,56.0,56.0
11427,Fnu,1.0,1.0
11428,Ausra,4.0,4.0
11429,Erin & Brad,82.0,82.0


In [54]:
clean_df.filter("number_of_reviews = 629 ").limit(5).toPandas()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,9145202,Room near JFK Queen Bed,47621202,Dona,Queens,Jamaica,40.667301,-73.768311,Private room,47,1,629,2019-01-05,14.58,2,333
